In [1]:
import gc
import io
import multiprocessing as mp
from copy import deepcopy
from urllib.parse import urlparse

import google.cloud.storage as gcs
import pandas as pd
import seaborn as sns
import Levenshtein
from retrying import retry
from tqdm.auto import tqdm

In [2]:
GCP_PROJECT = "dena-ai-training-28-gcp"


@retry(stop_max_attempt_number=3)
def download_from_gcs(path: str):
    url = urlparse(path)
    bucket_name = url.netloc
    storage_client = gcs.Client(project=GCP_PROJECT)
    bucket = storage_client.get_bucket(bucket_name)
    blob = gcs.Blob(url.path[1:], bucket)
    return blob.download_as_string()


def load_prediction(path: str):
    if path.split("/")[-2] == "kf-bms-candidates-v2":
        model = path.split("/")[-3]
    else:
        model = path.split("/")[-2]
    df = pd.read_csv(io.BytesIO(download_from_gcs(path)))\
        .assign(model=model, filename=path.split("/")[-1])
    return df

In [3]:
n_valid_InChIs = pd.read_csv(io.BytesIO(download_from_gcs("gs://kfujikawa-kaggle-bms-molecular-generation/kfujikawa/kf-bms-candidates-v2/test_n_valid_InChIs.csv")))
NO_VALID_IMAGE_IDs = n_valid_InChIs.query("n_valid_InChIs == 0").image_id
len(NO_VALID_IMAGE_IDs)

5152

In [4]:
CAMARO_TEST_CSVs = """
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_kf_0523.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_kf_0525.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_kf_0527.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_kf_0531.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_kf_0531_renormed.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_yokoo_0527.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_yokoo_0531.csv
gs://kfujikawa-kaggle-bms-molecular-generation/camaro/exp0845_v2/test_yokoo_0601.csv
""".strip().split()

In [5]:
with mp.Pool() as pool:
    total = len(CAMARO_TEST_CSVs)
    iterator = pool.imap_unordered(load_prediction, CAMARO_TEST_CSVs)
    camaro_df = pd.concat(list(tqdm(iterator, total=total)), ignore_index=True)
camaro_df = camaro_df.drop_duplicates(subset=["model", "image_id", "InChI"])
display(camaro_df.head(1))
display(camaro_df.groupby("model").image_id.count())

,image_id,InChI,levenshtein,is_valid,focal_score,ce_score,model,filename
0,001ae62e2309,InChI=1S/C11H11BrO4/c12-7-3-4-5(9(13)14)6-8(7)...,NaN,False,0.120724,0.559411,exp0845_v2,test_yokoo_0527.csv


model
exp0845_v2    5728274
Name: image_id, dtype: int64

In [7]:
sort_keys = dict(
    image_id=True,
    is_valid=False,
    focal_score=True,
)
camaro_ensembled_df = camaro_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby("image_id").first().reset_index()

In [8]:
baseline_df = pd.read_csv(io.BytesIO(download_from_gcs(
    "gs://kfujikawa-kaggle-bms-molecular-generation/submissions/LB059+kf0527.csv"
)))

In [11]:
merged_df = camaro_ensembled_df.merge(baseline_df, on="image_id")
merged_df["levenshtein"] = [
    Levenshtein.distance(x, y)
    for x, y in tqdm(merged_df[["InChI_x", "InChI_y"]].values)
]
print(merged_df.query("InChI_x != InChI_y").shape)
print(merged_df.levenshtein.mean())


(33625, 9)
0.21922693873458213


In [15]:
camaro_df.image_id.nunique()

1615577